In [3]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *


from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [2]:
desde = '2022-04-13'
hasta = '2022-07-13'
city = 'BOG'

In [3]:
query = """
WITH 
count_fulfillments_pasados AS (
	SELECT 
	    COUNT(DISTINCT boi.order_item_id) AS count_items,
	    bfg.order_id
     
	FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
	LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
	
	WHERE      
        ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status='SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
	GROUP BY 2
),

ful AS (

    SELECT DISTINCT
        bo.order_id,
        ffg.close_date
        
    FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg     ON bfg.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo     ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo      ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop     ON bop.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s       ON s.site_id = bo.site_disc
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba      ON ba.order_id = bo.order_id
    
    WHERE fo.fb_order_status_id IN (1,6,7,8)
        AND ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status = 'SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        
    GROUP BY 1,2
)

SELECT DISTINCT
    --bo.order_id,
    bo.customer_id,
    s.site_identifier_value as region_code,
    --TO_CHAR(ful.close_date, 'YYYY-mm-dd') AS day,
    bcat2.name as cat,
    bcat.name as subcat,
    --bs.sku_id,
    --bs.addl_product_id AS padre_sku_id,
    --bs.name AS product,
    SUM(boi.quantity) AS quantity,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
    ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd
    
FROM 
    postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
    INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
    LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
WHERE 
    ffg.close_date between '{desde}' AND '{hasta}'
    --AND s.site_identifier_value = {city}
    AND bo.order_status='SUBMITTED'
    AND fo.fb_order_status_id IN (1,6,7,8)
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND bcat2.name IN ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')

GROUP BY 1,2,3,4--,5--,6--,7,8--,9
""".format(desde=desde, hasta=hasta, city=city)
df = read_connection_data_warehouse.runQuery(query)

In [4]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
segmentos_full = get_fresh_query_result("https://internal-redash.federate.frubana.com/",101609,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [6]:
segmentos_full

,city,microsegment,customer_id,microsegment_id
0,CMX,Taquería en calle,63806119,15
1,BOG,Comida rápida,63807215,1
2,CMX,Fonda s/desayuno,63807326,14
3,SPO,Comida por quilo,63806754,20
4,SPO,Prato do dia c/café da manhã,63806270,19
...,...,...,...,...
113804,BOG,Panadería/Pastelería,185584744,6
113805,BOG,Comida rápida,185619623,1
113806,BOG,Parrilla/Asadero,185847384,5
113807,BOG,Panadería/Pastelería,186122765,6


In [83]:
#no_cats = ['Verduras', 'Frutas']
#df = df[~df.subcat.isin(no_cats)]

In [7]:
data_full = df.merge(segmentos_full[['customer_id','microsegment']], how = 'left', on = 'customer_id')
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [8]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique()

,customer_id
microsegment,
Cafetería,2843
Comida brasileira y saudavel,2083
Comida de mar y Colombiana,100
Comida especializada,2966
Comida internacional/Grill,725
Comida por quilo,2119
Comida rápida,5113
Corrientazo c/desayuno,1601
Corrientazo s/desayuno,7152


In [20]:
data_full['subcat'] = data_full['subcat'].str.strip()
data_full.dropna(axis=0, subset=['customer_id'], inplace=True)
data_full['customer_id'] = data_full['customer_id'].astype('str')

In [ ]:
#data_full[(data_full['region_code'] =="BOG") & (data_full['microsegment'].str.contains("Corrientazo", case=False))]

In [21]:
basket = (data_full[(data_full['region_code'] =="BOG") & (data_full['microsegment'].str.contains("Corrientazo", case=False))]
          .groupby(['customer_id', 'subcat'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('customer_id'))

In [22]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [23]:
basket_sets

subcat,Aceites & Grasas,Aguas,Alimentos Congelados,Arroz,Azúcar & Endulzantes,Bolsas,"Café, Chocolate e Infusiones",Cerdo Congelado,Cerdo Fresco,Cervezas & Licores,...,Pulpas de Fruta,Quesos,Res Congelado,Res Fresco,Sal & Sazonadores,Salsas,Tubérculos,Tés,Vasos,Verduras
customer_id,,,,,,,,,,,,,,,,,,,,,
-200051,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,1
-200079,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
-200511,1,0,0,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,1
-200579,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
-200611,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99797405,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
99832415,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
99833353,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
frequent_itemsets = fpgrowth(basket_sets, min_support=0.20, use_colnames=True)
frequent_itemsets

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


,support,itemsets
0,0.790538,(Verduras)
1,0.614390,(Contenedores)
2,0.608914,(Arroz)
3,0.483115,(Aceites & Grasas)
4,0.467752,(Vasos)
...,...,...
228,0.205202,"(Vasos, Granos, Verduras)"
229,0.212352,"(Detergente, Jabón & Lavalozas, Granos, Verduras)"
230,0.211591,"(Verduras, Sal & Sazonadores)"
231,0.241862,"(Verduras, Implementos de Aseo)"


In [25]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6) 
#support how many times of be together (P(AuC)) 
#confidence is equal to P(C/A)
#lift is to evaluate the support and confidence rull, there are times when both of these measures may be high, and yet still produce a rule that is not useful -> (Rule Support) /(Support(Antecedent) * Support(Consequent))
"""
Any rule with an improvement of less than 1 does not indicate a real cross-selling opportunity,
no matter how high its support and confidence, because it actually offers less ability to predict a purchase than does random chance.
"""

'\nAny rule with an improvement of less than 1 does not indicate a real cross-selling opportunity,\nno matter how high its support and confidence, because it actually offers less ability to predict a purchase than does random chance.\n'

In [26]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Contenedores),(Verduras),0.614390,0.790538,0.528445,0.860114,1.088010,0.042746,1.497372
1,(Verduras),(Contenedores),0.790538,0.614390,0.528445,0.668463,1.088010,0.042746,1.163096
2,(Arroz),(Verduras),0.608914,0.790538,0.519319,0.852860,1.078835,0.037949,1.423556
3,(Verduras),(Arroz),0.790538,0.608914,0.519319,0.656917,1.078835,0.037949,1.139918
4,(Arroz),(Contenedores),0.608914,0.614390,0.410100,0.673495,1.096201,0.035990,1.181023
...,...,...,...,...,...,...,...,...,...
545,"(Detergente, Jabón & Lavalozas, Granos)",(Verduras),0.224825,0.790538,0.212352,0.944520,1.194780,0.034619,3.775416
546,"(Granos, Verduras)","(Detergente, Jabón & Lavalozas)",0.330088,0.461363,0.212352,0.643318,1.394386,0.060061,1.510132
547,(Sal & Sazonadores),(Verduras),0.230149,0.790538,0.211591,0.919365,1.162961,0.029649,2.597666
548,(Implementos de Aseo),(Verduras),0.257986,0.790538,0.241862,0.937500,1.185901,0.037914,3.351384


In [100]:
rules.sort_values(by = 'confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,"(Tubérculos, Frutas)",(Verduras),0.264696,0.730611,0.257066,0.971173,1.329262,0.063676,9.345150
2,(Tubérculos),(Verduras),0.335339,0.730611,0.316210,0.942955,1.290639,0.071207,4.722383
40,"(Frutas, Detergente, Jabón & Lavalozas)",(Verduras),0.234426,0.730611,0.214902,0.916718,1.254728,0.043628,3.234648
66,(Implementos de Aseo),(Verduras),0.229232,0.730611,0.209851,0.915456,1.253000,0.042372,3.186362
63,"(Gaseosas, Vasos)",(Verduras),0.220168,0.730611,0.200931,0.912626,1.249128,0.040074,3.083177
...,...,...,...,...,...,...,...,...,...
54,(Contenedores),(Arroz),0.368906,0.468171,0.224324,0.608079,1.298839,0.051613,1.356980
42,(Vasos),(Arroz),0.391152,0.468171,0.237328,0.606741,1.295980,0.054202,1.352361
36,"(Detergente, Jabón & Lavalozas)","(Verduras, Papel & Toallas)",0.371736,0.399534,0.224109,0.602872,1.508936,0.075588,1.512020
12,(Arroz),(Aceites & Grasas),0.468171,0.433709,0.281390,0.601041,1.385814,0.078340,1.419419
